In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from random import random
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
import geopandas as gpd
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots
init_notebook_mode(connected=True)

In [ ]:
MLP_XKCD_COLOR = mlp.colors.XKCD_COLORS
MLP_BASE_COLOR = mlp.colors.BASE_COLORS
MLP_CNAMES = mlp.colors.cnames
MLP_CSS4 = mlp.colors.CSS4_COLORS
MLP_HEX = mlp.colors.hexColorPattern
MLP_TABLEAU = mlp.colors.TABLEAU_COLORS
print('I like COLORS :>')
def random_color_generator(color_type=None):
    if color_type is None:
        colors = sorted(MLP_CNAMES.items(), key=lambda x: random())
    else:
        colors = sorted(color_type.items(), key=lambda x: random())
    return dict(colors)

In [ ]:
path = '/kaggle/input/indian-food-101/indian_food.csv'
food = pd.read_csv(path)
food.tail(20)

In [ ]:
food.diet = food['diet'].replace('non vegetarian','non_vegetarian')

In [ ]:
def nans(df): return df[df.isnull().any(axis=1)]

In [ ]:
nans(food)

## Update value at index 110

In [ ]:
food.at[110,'region']= 'North'

In [ ]:
food[food['name']=='Panjeeri']

In [ ]:
food.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
labels,freq = np.unique(food.diet.values,return_counts=True)
colors = ['pink','lightgreen']
plt.pie(freq,labels=labels,autopct = '%1.1f%%',colors=colors)
plt.title('Diet Distribution')
plt.show()

In [ ]:
food[food['state']=='-1']

In [ ]:
plt.figure(figsize=(12,6))
colors = random_color_generator()
food['state'].value_counts().plot(kind='bar',color=colors)
plt.title('State wise Food Distribution ')
plt.show()

In [ ]:
colors = random_color_generator()
food.course.value_counts().plot(kind='bar',color=colors)
plt.title('Food Course Distribution')
plt.show()

In [ ]:
colors = random_color_generator()
food.region.value_counts().plot(kind='bar',color=colors)
plt.title('Region wise Food Distribution')
plt.show()

In [ ]:
colors = random_color_generator(MLP_CSS4)
food.flavor_profile.value_counts().plot(kind='bar',color=colors)
plt.title('Flavor wise Food Distribution')
plt.show()

In [ ]:
def fillna(col,typeX=None):
    if typeX:
        full[col].fillna('0',inplace=True)
    else:
        full[col].fillna('Unknown',inplace=True)
    return full[col]

In [ ]:
shp_gdf = gpd.read_file('../input/india-gis-data/India States/Indian_states.shp')

In [ ]:
food_diet = food[food['diet']=='vegetarian']['state'].value_counts()
food_diet_df = food_diet.reset_index()
food_diet_df.columns = ['st_nm','count']
merged = pd.merge(shp_gdf, food_diet_df, on='st_nm', how='outer')
merged['count'].fillna('0',inplace=True)
fig, ax = plt.subplots(1, figsize=(10, 10))
ax.axis('off')
ax.set_title('State-wise Distribution of Indian Vegetarian Diet ',
             fontdict={'fontsize': '15', 'fontweight' : '3'})
fig = merged.plot(column='count', cmap='Greens', linewidth=0.5, ax=ax, edgecolor='0.2',legend=True)

In [ ]:
food_diet = food[food['diet']!='vegetarian']['state'].value_counts()
food_diet_df = food_diet.reset_index()
food_diet_df.columns = ['st_nm','count']
merged = pd.merge(shp_gdf, food_diet_df, on='st_nm', how='outer')
merged['count'].fillna('0',inplace=True)
fig, ax = plt.subplots(1, figsize=(10, 10))
ax.axis('off')
ax.set_title('State-wise Distribution of Indian Non-Vegetarian Diet ',
             fontdict={'fontsize': '15', 'fontweight' : '3'})
fig = merged.plot(column='count', cmap='OrRd', linewidth=0.5, ax=ax, edgecolor='0.2',legend=True)

In [ ]:
def distribute_flavors(flavor,cmap):
    food_flvor = food[food['flavor_profile']==flavor]['state'].value_counts()
    food_flvor_df = food_flvor.reset_index()
    food_flvor_df.columns = ['st_nm','count']
    #print(food_flvor_df)
    merged = pd.merge(shp_gdf, food_flvor_df, on='st_nm', how='outer')
    merged['count'].fillna('0',inplace=True)
    fig, ax = plt.subplots(1, figsize=(10, 10))
    ax.axis('off')
    ax.set_title('State-wise Distribution of Indian '+flavor,
                 fontdict={'fontsize': '15', 'fontweight' : '3'})
    fig = merged.plot(column='count', cmap=cmap, linewidth=0.5, ax=ax, edgecolor='0.2',legend=True)

In [ ]:
distribute_flavors('spicy','gist_heat')

In [ ]:
distribute_flavors('sweet','Spectral')

In [ ]:
distribute_flavors('bitter','seismic')

In [ ]:
distribute_flavors('sour','Pastel1')

In [ ]:
distribute_flavors('-1','jet')

In [ ]:
colors=random_color_generator()
x = food.groupby('diet')['course'].value_counts().unstack()
#print(x)
x.plot.bar(stacked=True)
plt.title('Diet Vs Course Distribution')
plt.plot()

In [ ]:
colors=random_color_generator()
food.groupby('region')['diet'].value_counts().unstack().plot.bar(stacked=True,color=list(colors.keys()))
plt.title('Food Distribution')
plt.plot()

In [ ]:
sweet_food = food[food['flavor_profile']=='sweet']
sweet_food#.prep_time.sum()/88

In [ ]:
course_food = food[food['course']=='main course']
course_food['diet'].value_counts().values

In [ ]:
def food_infograph_course(course):
    course_food = food[food['course']==course]
    total_course_food= course_food.shape[0]
    avg_prep_time = course_food.prep_time.sum()/total_course_food
    avg_cook_time = course_food.cook_time.sum()/total_course_food

    fig = make_subplots(
        rows=1, cols=4,subplot_titles=('Total Dishes','Avg Prep Time','Avg Cook Time','Diet Ratio'),
        specs=[[{'type': 'indicator'},{'type': 'indicator'},{'type': 'indicator'},{'type': 'pie'}]])

    fig.add_trace(go.Indicator(
        mode = 'number',
        value = int(total_course_food),
        number={'font':{'color': 'blue','size':50}},
    ),row=1, col=1)
    fig.add_trace(go.Indicator(
        mode = 'number',
        value = int(avg_prep_time),
        number={'font':{'color': 'green','size':50}},
    ),row=1, col=2)
    fig.add_trace(go.Indicator(
        mode = 'number',
        value = int(avg_cook_time),
        number={'font':{'color': 'red','size':50}},
    ),row=1, col=3)

    fig.add_trace(go.Pie(labels=course_food['diet'].value_counts().index, 
                     values=course_food['diet'].value_counts().values,textinfo='percent',
                    marker= dict(colors=['green','red'])),row=1, col=4)
    
    fig.update_layout(
        title_text= course+' Food Infograph',template='plotly',
        title_x=0.5)
    fig.show()

In [ ]:
food_infograph_course('main course')

In [ ]:
food_infograph_course('dessert')

In [ ]:
food_infograph_course('snack')

In [ ]:
food_infograph_course('starter')